In [1]:
%matplotlib inline

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [3]:
#To determine if your system supports CUDA
print("==> Check devices..")
if torch.cuda.is_available():
    device = 'cuda'
else:    
    device = 'cpu'
print("Current device: ",device)
if device == 'cuda':
    print("Our selected device: ", torch.cuda.current_device())
    print(torch.cuda.device_count(), " GPUs is available")

==> Check devices..
Current device:  cuda
Our selected device:  0
1  GPUs is available


In [4]:
def get_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for inputs, targets in dataloader:
        for i in range(3):
            mean[i] += inputs[:, i, :, :].mean()
            std[i] += inputs[:, i, :, :].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

In [5]:
calculate_mean_std = False

if calculate_mean_std == True:
    #we will calculate mean and std
    #The transform function for train data
    transform_train = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])

    transform_val = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])

    #The transform function for test data
    transform_test = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])

    trainset = torchvision.datasets.ImageFolder(root='../data/cinic10/train', transform=transform_train)
    valset = torchvision.datasets.ImageFolder(root='../data/cinic10/valid', transform=transform_val)
    testset = torchvision.datasets.ImageFolder(root='../data/cinic10/test', transform=transform_test)

    train_mean, train_std = get_mean_and_std(trainset)
    print(train_mean, train_std)
    val_mean, val_std = get_mean_and_std(valset)
    print(val_mean, val_std)
    test_mean, test_std = get_mean_and_std(testset)
    print(test_mean, test_std)
else:
    train_mean, train_std = ([0.4770, 0.4667, 0.4244]), ([0.1884, 0.1852, 0.1874])
    val_mean, val_std = ([0.4769, 0.4663, 0.4240]), ([0.1885, 0.1850, 0.1872])
    test_mean, test_std = ([0.4769, 0.4668, 0.4245]), ([0.1886, 0.1851, 0.1874])

In [6]:
#The transform function for train data
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation((-180,180)),
    transforms.RandomAffine(degrees=(-30,30), shear=(-20,20)),
    #transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std)
])

#The transform function for validation data
transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(val_mean, val_std)
])

#The transform function for test data
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(test_mean, test_std)
])

In [7]:
trainset = torchvision.datasets.ImageFolder(root='../data/cinic10/train', transform=transform_train)
valset = torchvision.datasets.ImageFolder(root='../data/cinic10/valid', transform=transform_val)
testset = torchvision.datasets.ImageFolder(root='../data/cinic10/test', transform=transform_test)

In [8]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
    shuffle=True, num_workers=2)

valloader = torch.utils.data.DataLoader(valset, batch_size=32,
    shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=32,
    shuffle=False, num_workers=2)

In [9]:
classes = ('airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [10]:
import torch.nn as nn

In [11]:
model = torchvision.models.resnet18(pretrained=False)
if device == 'cuda':
    model = model.cuda(0)
else:
    model = model.cpu()

In [12]:
import torch.optim as optim

#loss function
criterion = nn.CrossEntropyLoss()
#optimization algorithm
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.001)

In [13]:
for phase in ['train', 'val']:
    if phase == 'train':
        print(phase)    
    else:
        print(phase)

train
val


In [14]:
import copy

best_model = model
best_acc = 0.0
val_acc = 0.0
print(device)

model.train()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, verbose=True)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma=0.5)

for epoch in range(200):  # loop over the dataset multiple times
        
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train(True)  # Set model to training mode
            dset_loaders = trainloader
        else:
            model.train(False)  # Set model to evaluate mode
            dset_loaders = valloader
            
        running_loss = 0.0
        correct = 0
        
        for i, data in enumerate(dset_loaders, 0):
        
            (inputs, labels) = data
            
            #change the type into cuda tensor 
            if device == 'cuda':
                inputs = inputs.cuda(0)
                labels = labels.cuda(0)
            else:
                inputs = inputs.cpu()
                labels = labels.cpu()

            #print(labels)
            #print(inputs)
        
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            # select the class with highest probability
            _, pred = outputs.max(1)
            # if the model predicts the same results as the true
            # label, then the correct counter will plus 1
            correct += pred.eq(labels).sum().item()
        
            loss = criterion(outputs, labels)
        
            if phase == 'train':
                loss.backward()
                optimizer.step()

            # print statistics
            running_loss += loss.item()
            if (i % 200) == 199:    # print every 200 mini-batches
                print('[%d, %5d]' % (epoch + 1, i + 1))
            
        if phase == 'train':
            epoch_loss = running_loss / len(trainset)
            epoch_acc = correct / len(trainset)
        else:
            epoch_loss = running_loss / len(valset)
            epoch_acc = correct / len(valset)
            val_acc = epoch_acc

        print('{:d} Epoch {} Loss: {:.4f} Acc: {:.4f}'.format(epoch, phase, epoch_loss, epoch_acc))

        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model)
            torch.save(best_model, './model.pt')
    lr_scheduler.step(val_acc)
print('Finished Training')

model = best_model

cuda
[1,   200]
[1,   400]
[1,   600]
[1,   800]
[1,  1000]
[1,  1200]
[1,  1400]
[1,  1600]
[1,  1800]
[1,  2000]
[1,  2200]
[1,  2400]
[1,  2600]
[1,  2800]
0 Epoch train Loss: 0.0682 Acc: 0.1955
[1,   200]
[1,   400]
[1,   600]
[1,   800]
[1,  1000]
[1,  1200]
[1,  1400]
[1,  1600]
[1,  1800]
[1,  2000]
[1,  2200]
[1,  2400]
[1,  2600]
[1,  2800]
0 Epoch val Loss: 0.0629 Acc: 0.2383
[2,   200]
[2,   400]
[2,   600]
[2,   800]
[2,  1000]
[2,  1200]
[2,  1400]
[2,  1600]
[2,  1800]
[2,  2000]
[2,  2200]
[2,  2400]
[2,  2600]
[2,  2800]
1 Epoch train Loss: 0.0649 Acc: 0.2216
[2,   200]
[2,   400]
[2,   600]
[2,   800]
[2,  1000]
[2,  1200]
[2,  1400]
[2,  1600]
[2,  1800]
[2,  2000]
[2,  2200]
[2,  2400]
[2,  2600]
[2,  2800]
1 Epoch val Loss: 0.0626 Acc: 0.2289
[3,   200]
[3,   400]
[3,   600]
[3,   800]
[3,  1000]
[3,  1200]
[3,  1400]
[3,  1600]
[3,  1800]
[3,  2000]
[3,  2200]
[3,  2400]
[3,  2600]
[3,  2800]
2 Epoch train Loss: 0.0639 Acc: 0.2388
[3,   200]
[3,   400]
[3,   600]
[

[21,  2200]
[21,  2400]
[21,  2600]
[21,  2800]
20 Epoch train Loss: 0.0567 Acc: 0.3362
[21,   200]
[21,   400]
[21,   600]
[21,   800]
[21,  1000]
[21,  1200]
[21,  1400]
[21,  1600]
[21,  1800]
[21,  2000]
[21,  2200]
[21,  2400]
[21,  2600]
[21,  2800]
20 Epoch val Loss: 0.0531 Acc: 0.3816
[22,   200]
[22,   400]
[22,   600]
[22,   800]
[22,  1000]
[22,  1200]
[22,  1400]
[22,  1600]
[22,  1800]
[22,  2000]
[22,  2200]
[22,  2400]
[22,  2600]
[22,  2800]
21 Epoch train Loss: 0.0565 Acc: 0.3391
[22,   200]
[22,   400]
[22,   600]
[22,   800]
[22,  1000]
[22,  1200]
[22,  1400]
[22,  1600]
[22,  1800]
[22,  2000]
[22,  2200]
[22,  2400]
[22,  2600]
[22,  2800]
21 Epoch val Loss: 0.0545 Acc: 0.3574
[23,   200]
[23,   400]
[23,   600]
[23,   800]
[23,  1000]
[23,  1200]
[23,  1400]
[23,  1600]
[23,  1800]
[23,  2000]
[23,  2200]
[23,  2400]
[23,  2600]
[23,  2800]
22 Epoch train Loss: 0.0562 Acc: 0.3443
[23,   200]
[23,   400]
[23,   600]
[23,   800]
[23,  1000]
[23,  1200]
[23,  1400]


[40,  2200]
[40,  2400]
[40,  2600]
[40,  2800]
39 Epoch val Loss: 0.0409 Acc: 0.5254
[41,   200]
[41,   400]
[41,   600]
[41,   800]
[41,  1000]
[41,  1200]
[41,  1400]
[41,  1600]
[41,  1800]
[41,  2000]
[41,  2200]
[41,  2400]
[41,  2600]
[41,  2800]
40 Epoch train Loss: 0.0458 Acc: 0.4736
[41,   200]
[41,   400]
[41,   600]
[41,   800]
[41,  1000]
[41,  1200]
[41,  1400]
[41,  1600]
[41,  1800]
[41,  2000]
[41,  2200]
[41,  2400]
[41,  2600]
[41,  2800]
40 Epoch val Loss: 0.0411 Acc: 0.5271
[42,   200]
[42,   400]
[42,   600]
[42,   800]
[42,  1000]
[42,  1200]
[42,  1400]
[42,  1600]
[42,  1800]
[42,  2000]
[42,  2200]
[42,  2400]
[42,  2600]
[42,  2800]
41 Epoch train Loss: 0.0457 Acc: 0.4747
[42,   200]
[42,   400]
[42,   600]
[42,   800]
[42,  1000]
[42,  1200]
[42,  1400]
[42,  1600]
[42,  1800]
[42,  2000]
[42,  2200]
[42,  2400]
[42,  2600]
[42,  2800]
41 Epoch val Loss: 0.0403 Acc: 0.5418
[43,   200]
[43,   400]
[43,   600]
[43,   800]
[43,  1000]
[43,  1200]
[43,  1400]
[4

[60,  2200]
[60,  2400]
[60,  2600]
[60,  2800]
59 Epoch train Loss: 0.0392 Acc: 0.5539
[60,   200]
[60,   400]
[60,   600]
[60,   800]
[60,  1000]
[60,  1200]
[60,  1400]
[60,  1600]
[60,  1800]
[60,  2000]
[60,  2200]
[60,  2400]
[60,  2600]
[60,  2800]
59 Epoch val Loss: 0.0338 Acc: 0.6173
[61,   200]
[61,   400]
[61,   600]
[61,   800]
[61,  1000]
[61,  1200]
[61,  1400]
[61,  1600]
[61,  1800]
[61,  2000]
[61,  2200]
[61,  2400]
[61,  2600]
[61,  2800]
60 Epoch train Loss: 0.0389 Acc: 0.5572
[61,   200]
[61,   400]
[61,   600]
[61,   800]
[61,  1000]
[61,  1200]
[61,  1400]
[61,  1600]
[61,  1800]
[61,  2000]
[61,  2200]
[61,  2400]
[61,  2600]
[61,  2800]
60 Epoch val Loss: 0.0340 Acc: 0.6159
[62,   200]
[62,   400]
[62,   600]
[62,   800]
[62,  1000]
[62,  1200]
[62,  1400]
[62,  1600]
[62,  1800]
[62,  2000]
[62,  2200]
[62,  2400]
[62,  2600]
[62,  2800]
61 Epoch train Loss: 0.0388 Acc: 0.5571
[62,   200]
[62,   400]
[62,   600]
[62,   800]
[62,  1000]
[62,  1200]
[62,  1400]


[80,   200]
[80,   400]
[80,   600]
[80,   800]
[80,  1000]
[80,  1200]
[80,  1400]
[80,  1600]
[80,  1800]
[80,  2000]
[80,  2200]
[80,  2400]
[80,  2600]
[80,  2800]
79 Epoch train Loss: 0.0366 Acc: 0.5865
[80,   200]
[80,   400]
[80,   600]
[80,   800]
[80,  1000]
[80,  1200]
[80,  1400]
[80,  1600]
[80,  1800]
[80,  2000]
[80,  2200]
[80,  2400]
[80,  2600]
[80,  2800]
79 Epoch val Loss: 0.0317 Acc: 0.6434
[81,   200]
[81,   400]
[81,   600]
[81,   800]
[81,  1000]
[81,  1200]
[81,  1400]
[81,  1600]
[81,  1800]
[81,  2000]
[81,  2200]
[81,  2400]
[81,  2600]
[81,  2800]
80 Epoch train Loss: 0.0366 Acc: 0.5851
[81,   200]
[81,   400]
[81,   600]
[81,   800]
[81,  1000]
[81,  1200]
[81,  1400]
[81,  1600]
[81,  1800]
[81,  2000]
[81,  2200]
[81,  2400]
[81,  2600]
[81,  2800]
80 Epoch val Loss: 0.0316 Acc: 0.6441
[82,   200]
[82,   400]
[82,   600]
[82,   800]
[82,  1000]
[82,  1200]
[82,  1400]
[82,  1600]
[82,  1800]
[82,  2000]
[82,  2200]
[82,  2400]
[82,  2600]
[82,  2800]
81 E

[99,  1200]
[99,  1400]
[99,  1600]
[99,  1800]
[99,  2000]
[99,  2200]
[99,  2400]
[99,  2600]
[99,  2800]
98 Epoch val Loss: 0.0314 Acc: 0.6465
[100,   200]
[100,   400]
[100,   600]
[100,   800]
[100,  1000]
[100,  1200]
[100,  1400]
[100,  1600]
[100,  1800]
[100,  2000]
[100,  2200]
[100,  2400]
[100,  2600]
[100,  2800]
99 Epoch train Loss: 0.0362 Acc: 0.5893
[100,   200]
[100,   400]
[100,   600]
[100,   800]
[100,  1000]
[100,  1200]
[100,  1400]
[100,  1600]
[100,  1800]
[100,  2000]
[100,  2200]
[100,  2400]
[100,  2600]
[100,  2800]
99 Epoch val Loss: 0.0315 Acc: 0.6453
[101,   200]
[101,   400]
[101,   600]
[101,   800]
[101,  1000]
[101,  1200]
[101,  1400]
[101,  1600]
[101,  1800]
[101,  2000]
[101,  2200]
[101,  2400]
[101,  2600]
[101,  2800]
100 Epoch train Loss: 0.0361 Acc: 0.5922
[101,   200]
[101,   400]
[101,   600]
[101,   800]
[101,  1000]
[101,  1200]
[101,  1400]
[101,  1600]
[101,  1800]
[101,  2000]
[101,  2200]
[101,  2400]
[101,  2600]
[101,  2800]
100 Epo

[117,  2600]
[117,  2800]
116 Epoch val Loss: 0.0314 Acc: 0.6468
[118,   200]
[118,   400]
[118,   600]
[118,   800]
[118,  1000]
[118,  1200]
[118,  1400]
[118,  1600]
[118,  1800]
[118,  2000]
[118,  2200]
[118,  2400]
[118,  2600]
[118,  2800]
117 Epoch train Loss: 0.0361 Acc: 0.5908
[118,   200]
[118,   400]
[118,   600]
[118,   800]
[118,  1000]
[118,  1200]
[118,  1400]
[118,  1600]
[118,  1800]
[118,  2000]
[118,  2200]
[118,  2400]
[118,  2600]
[118,  2800]
117 Epoch val Loss: 0.0313 Acc: 0.6473
[119,   200]
[119,   400]
[119,   600]
[119,   800]
[119,  1000]
[119,  1200]
[119,  1400]
[119,  1600]
[119,  1800]
[119,  2000]
[119,  2200]
[119,  2400]
[119,  2600]
[119,  2800]
118 Epoch train Loss: 0.0361 Acc: 0.5911
[119,   200]
[119,   400]
[119,   600]
[119,   800]
[119,  1000]
[119,  1200]
[119,  1400]
[119,  1600]
[119,  1800]
[119,  2000]
[119,  2200]
[119,  2400]
[119,  2600]
[119,  2800]
118 Epoch val Loss: 0.0314 Acc: 0.6475
[120,   200]
[120,   400]
[120,   600]
[120,   

Process Process-513:
Process Process-514:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/root/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/root/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/root/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/root/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/root/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/root/anaconda3/lib/python3.7/site-packages/torch/utils/data/dat

KeyboardInterrupt: 

In [56]:
print('==> Saving model..')

#only save model parameters
torch.save(model.state_dict(), './checkpoint.t7')
#you also can store some log information
state = {
    'net': model.state_dict(),
    'acc': 100.*correct/len(trainset),
    'epoch': epoch
}
torch.save(state, './checkpoint.t7')

#save entire model
torch.save(model, './model.pt')

print('Finished Saving')

==> Saving model..
Finished Saving


In [57]:
print('==> Testing model..')
model.eval()

==> Testing model..


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [58]:
correct = 0
total = 0
running_loss = 0.0
# device == 'cpu'
class_correct = list(0. for i in range(11))
class_total = list(0. for i in range(11))

with torch.no_grad():
    for mini_batch, data in enumerate(testloader, 0):
        images, labels = data

        if device == 'cuda':
            images = images.cuda(0)
            labels = labels.cuda(0)
        else:
            images = images.cpu()
            labels = labels.cpu()
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
        running_loss += loss.item()

        c = (predicted == labels).squeeze()
        #print(predicted)
        #print(labels.size(0))
        #print(c)
        for i in range(labels.size(0)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
        if(mini_batch % 200 == 199):
            print('[%5d]' % (mini_batch + 1))
            
print('Accuracy of the network on the %d test images: %.2f%%, and loss is: %.3f'
      % (total, 100 * correct / total, running_loss / total))

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

[  200]
[  400]
[  600]
[  800]
[ 1000]
[ 1200]
[ 1400]
[ 1600]
[ 1800]
[ 2000]
[ 2200]
[ 2400]
[ 2600]
[ 2800]
Accuracy of the network on the 90000 test images: 70.05%, and loss is: 0.036
Accuracy of airplane : 81 %
Accuracy of automobile : 75 %
Accuracy of  bird : 62 %
Accuracy of   cat : 57 %
Accuracy of  deer : 60 %
Accuracy of   dog : 53 %
Accuracy of  frog : 82 %
Accuracy of horse : 75 %
Accuracy of  ship : 78 %
Accuracy of truck : 73 %
